In [15]:
import numpy as np

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [16]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/qrpfra_v3_leg_ik_scene_right.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):

        self.do_simulation(action, self.frame_skip)

        observation = self._get_obs()
        reward = 0
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [17]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([ 0.046 ,  0.1435, -0.1245,  0.    ,  0.    ,  0.    ]), {'works': True})


/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:247: UserWarning: WARN: For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:225: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:229: UserWarning: WARN: A Box observation space maximum value is -infinity. This is probably too high.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:321: UserWarning: WARN: Not able to test alternative render modes due to the environ

In [18]:
leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [19]:
for i in range(-100, 101):
    for j in range(-100, 101):
        for k in range(-100, 101):
            #env.render_mode = "human"
            env_obs = []
            for step_cnt in range(0,3):
                obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
                if step_cnt == 2:
                    env_obs = obs
            

            leg_dataset.append([env_obs[0], env_obs[1], env_obs[2], i/100, j/100, k/100])


env.close()

In [20]:
print(len(leg_dataset))

8120601


In [21]:

# Save the leg_dataset variable to a text file with comma separation
np.savetxt('leg_dataset.txt', leg_dataset, delimiter=',')


In [22]:
#Convert left leg list to numpy array
leg_dataset_as_np = np.array(leg_dataset)

In [23]:
print(leg_dataset_as_np[0:4])

observations = leg_dataset_as_np[:, 0:3][:]
print(observations)
actions = leg_dataset_as_np[:, 3:][:]
print(actions)

[[ 0.18159074 -0.03761695 -0.15039326 -1.         -1.         -1.        ]
 [ 0.18023417 -0.16109168 -0.04791185 -1.         -1.         -0.99      ]
 [ 0.18332317 -0.15551806 -0.05289781 -1.         -1.         -0.98      ]
 [ 0.18468985 -0.15283653 -0.05502772 -1.         -1.         -0.97      ]]
[[ 0.18159074 -0.03761695 -0.15039326]
 [ 0.18023417 -0.16109168 -0.04791185]
 [ 0.18332317 -0.15551806 -0.05289781]
 ...
 [ 0.07799425  0.04113232 -0.01014912]
 [ 0.0774545   0.03991276 -0.01046013]
 [ 0.07693895  0.03877661 -0.01075719]]
[[-1.   -1.   -1.  ]
 [-1.   -1.   -0.99]
 [-1.   -1.   -0.98]
 ...
 [ 1.    1.    0.98]
 [ 1.    1.    0.99]
 [ 1.    1.    1.  ]]


In [24]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1, shuffle=True)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(3,)),
    tf.keras.layers.Dense(3, activation='linear'),
    tf.keras.layers.Dense(32, activation='linear'),
    tf.keras.layers.Dense(64, activation='linear'),
    tf.keras.layers.Dense(32, activation='linear'),
    tf.keras.layers.Dense(3, activation='tanh')
])
# Compile the model
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train the model
model.fit(observations, actions, epochs=5, batch_size=256)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)


Epoch 1/5
31722/31722 [==============================] - 26s 817us/step - loss: 0.0189 - accuracy: 0.9435
Epoch 2/5
31722/31722 [==============================] - 26s 808us/step - loss: 0.0150 - accuracy: 0.9548
Epoch 3/5
31722/31722 [==============================] - 25s 792us/step - loss: 0.0145 - accuracy: 0.9580
Epoch 4/5
31722/31722 [==============================] - 23s 717us/step - loss: 0.0142 - accuracy: 0.9604
Epoch 5/5
25377/25377 [==============================] - 7s 293us/step - loss: 0.0138 - accuracy: 0.9634
Test Loss: [0.0138414790853858, 0.9633870124816895]


In [25]:
model.save("right_legs_model", overwrite=True)

INFO:tensorflow:Assets written to: right_legs_model/assets


INFO:tensorflow:Assets written to: right_legs_model/assets
